In [311]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import fuzzywuzzy
from fuzzywuzzy import process
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/vehicle-sales-data/car_prices.csv


# Setup
Importing data

In [312]:
vehicle_location = '../input/vehicle-sales-data/car_prices.csv'
vehicle_data = pd.read_csv(vehicle_location)

# Showing table top 5 rows

In [313]:
vehicle_data.head()

,year,make,model,trim,body,transmission,vin,state,condition,odometer,color,interior,seller,mmr,sellingprice,saledate
0,2015,Kia,Sorento,LX,SUV,automatic,5xyktca69fg566472,ca,5.0,16639.0,white,black,kia motors america inc,20500.0,21500.0,Tue Dec 16 2014 12:30:00 GMT-0800 (PST)
1,2015,Kia,Sorento,LX,SUV,automatic,5xyktca69fg561319,ca,5.0,9393.0,white,beige,kia motors america inc,20800.0,21500.0,Tue Dec 16 2014 12:30:00 GMT-0800 (PST)
2,2014,BMW,3 Series,328i SULEV,Sedan,automatic,wba3c1c51ek116351,ca,45.0,1331.0,gray,black,financial services remarketing (lease),31900.0,30000.0,Thu Jan 15 2015 04:30:00 GMT-0800 (PST)
3,2015,Volvo,S60,T5,Sedan,automatic,yv1612tb4f1310987,ca,41.0,14282.0,white,black,volvo na rep/world omni,27500.0,27750.0,Thu Jan 29 2015 04:30:00 GMT-0800 (PST)
4,2014,BMW,6 Series Gran Coupe,650i,Sedan,automatic,wba6b2c57ed129731,ca,43.0,2641.0,gray,black,financial services remarketing (lease),66000.0,67000.0,Thu Dec 18 2014 12:30:00 GMT-0800 (PST)


# Droping useless columns

In [314]:
vehicle_data_formated = vehicle_data
vehicle_data_formated.drop(labels = ['interior','seller','trim','saledate','mmr','state'],axis = 1,inplace = True)

# Formating data

## Cleaning transmission

In [315]:
#function to find transmission in columns 
def find_transmission(row):
    for record in row:
        if record == 'automatic' or record == 'manual':
            return record
    return row['transmission']
#apply function and changing data
vehicle_data_formated['transmission'] = vehicle_data.apply(find_transmission, axis=1)
#checking what left
wrong_transmision = vehicle_data_formated[(vehicle_data_formated['transmission'] != 'automatic') & (vehicle_data_formated['transmission'] != 'manual') & (
vehicle_data_formated['transmission'].isna() !=True)]
wrong_transmision

,year,make,model,body,transmission,vin,condition,odometer,color,sellingprice
461612,2015,Volkswagen,Jetta,Navitgation,Sedan,NaN,NaN,46.0,2711,14250.0
505299,2015,Volkswagen,Jetta,Navitgation,sedan,NaN,NaN,36.0,20379,13600.0
529009,2015,Volkswagen,Jetta,Navitgation,sedan,NaN,NaN,2.0,2817,13750.0
551222,2015,Volkswagen,Jetta,Navitgation,sedan,NaN,NaN,2.0,9562,13200.0


In [316]:
# i'll delete it cuz there is a lot of missing data
# creating index_transmision_to_delete
index_transmision_to_delete = wrong_transmision.index
#deleting rows
vehicle_data_formated.drop(index_transmision_to_delete, inplace=True)

## Cleaning vin

In [317]:
def find_vin(row):
    for record in row:
        try:
            if len(record) == 17:
                return record
        except:
            pass
    return row['vin']
vehicle_data_formated['vin'] = vehicle_data.apply(find_vin, axis = 1)

In [318]:
vehicle_data_formated.sample(5)

,year,make,model,body,transmission,vin,condition,odometer,color,sellingprice
555819,2010,BMW,6 Series,coupe,automatic,wbaea5c53acv93163,42.0,47665.0,black,31250.0
107505,2014,Honda,CR-V,SUV,automatic,5j6rm3h34el032347,49.0,5001.0,red,20900.0
74744,2012,Honda,Pilot,SUV,NaN,5fnyf4h72cb005572,44.0,24615.0,gray,26000.0
373808,2014,Audi,A4,Sedan,automatic,wauffafl5en006349,43.0,16979.0,silver,32000.0
182248,2014,Ford,Expedition,SUV,automatic,1fmju1k50eef56371,46.0,14737.0,gray,31500.0


In [328]:
# Checking what left
left_vin =  vehicle_data_formated[vehicle_data_formated.vin.str.len() != 17]
left_vin

,year,make,model,body,transmission,vin,condition,odometer,color,sellingprice


In [320]:
# Deleting whats left 
vehicle_data_formated.drop(left_vin.index, inplace = True)

## Cleaning car makers

In [322]:
vehicle_data_formated = vehicle_data_formated[vehicle_data_formated['make'].isnull() == False]
car_makers = vehicle_data_formated.make.unique()
np.unique(np.array([car.lower() for car in car_makers]))

array(['acura', 'airstream', 'aston martin', 'audi', 'bentley', 'bmw',
       'buick', 'cadillac', 'chev truck', 'chevrolet', 'chrysler',
       'daewoo', 'dodge', 'dodge tk', 'dot', 'ferrari', 'fiat', 'fisker',
       'ford', 'ford tk', 'ford truck', 'geo', 'gmc', 'gmc truck',
       'honda', 'hummer', 'hyundai', 'hyundai tk', 'infiniti', 'isuzu',
       'jaguar', 'jeep', 'kia', 'lamborghini', 'land rover', 'landrover',
       'lexus', 'lincoln', 'lotus', 'maserati', 'mazda', 'mazda tk',
       'mercedes', 'mercedes-b', 'mercedes-benz', 'mercury', 'mini',
       'mitsubishi', 'nissan', 'oldsmobile', 'plymouth', 'pontiac',
       'porsche', 'ram', 'rolls-royce', 'saab', 'saturn', 'scion',
       'smart', 'subaru', 'suzuki', 'tesla', 'toyota', 'volkswagen',
       'volvo', 'vw'], dtype='<U13')

### Cleaning car makers 

In [323]:
def formating_car_makers(brand):
    if isinstance(brand, str):
        mapping = {
            'chev truck': 'chevrolet',
            'dodge tk': 'dodge',
            'gmc truck': 'gmc',
            'hyundai tk': 'hyundai',
            'land rover': 'landrover',
            'mazda tk': 'mazda',
            'mercedes-b': 'mercedes',
            'mercedes-benz': 'mercedes',
            'vw': 'volkswagen'
        }
        brand = brand.lower()
        if brand in mapping:
            return mapping[brand]
        else:
            return brand
    else:
        ''
    
vehicle_data_formated.make = vehicle_data.make.apply(lambda brand: formating_car_makers(brand))
vehicle_data_formated = vehicle_data_formated[vehicle_data_formated['make'] != 'dot']

/tmp/ipykernel_33/1626176101.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vehicle_data_formated.make = vehicle_data.make.apply(lambda brand: formating_car_makers(brand))


## Cleaning car models

In [324]:
vehicle_data_formated.sample(5)

,year,make,model,body,transmission,vin,condition,odometer,color,sellingprice
94619,2002,dodge,Grand Caravan,Minivan,automatic,2b4gp74l72r520502,19.0,228030.0,gold,1150.0
29673,1998,pontiac,Sunfire,Coupe,automatic,1g2jb1246w7570058,NaN,NaN,orange,400.0
109075,2011,mercedes,C-Class,Sedan,automatic,wddgf5ebxbr176756,35.0,33641.0,black,18750.0
263487,2012,audi,A4,Sedan,automatic,waubfafl4cn015118,26.0,34475.0,white,16400.0
431721,2013,honda,Odyssey,Minivan,automatic,5fnrl5h2xdb028557,34.0,23910.0,white,16900.0


In [325]:
vehicle_data_formated.model = vehicle_data_formated.model.str.lower()
vehicle_data_formated.model = vehicle_data_formated.model.str.strip()


## Cleaning body

In [326]:
vehicle_data_formated.body = vehicle_data_formated.body.str.lower()
vehicle_data_formated.body = vehicle_data_formated.body.str.strip()


In [327]:
vehicle_data_formated.transmission.unique()

array(['automatic', nan, 'manual'], dtype=object)